In [1]:
import pandas as pd
import os
import warnings
from statsmodels.formula.api import ols
import statsmodels.api as sm
warnings.filterwarnings('ignore')  # Suppress all warnings
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

### Load the data

In [3]:
# Reload the updated data
file_path = "data/2025-07-10-LE_Lisket_0_15_AMBI_DEF.xlsx"
df = pd.read_excel(file_path, sheet_name='LE_Lisket_0_15_AMBI')

In [4]:
# Drop rows with missing values in Group_Sex or Year
df = df.dropna(subset=['GR_Gender', 'Year'])
# Standardize Group_Sex values to lowercase
df['Sex'] = df['Gender'].str.lower()
# Standardize Group_Sex values to lowercase
df['Group_Sex'] = df['GR_Gender'].str.lower()

### ANOVA test

In [6]:
# Define variables to test and corresponding feature names in the dataset
anova_targets = {
    'Loco_TOT': 'Locomotion (Loco_TOT)',
    'Loco_BEF_Calc': 'Locomotion frequency (Loco_BEF_Calc)',
    'Expl_TOT_Nr': 'Exploration (Expl_TOT_Nr)',
    'Expl_E_I_BEF_Nr': 'Exploration frequency (Expl_E_I_BEF_Nr)',
    'L_C_ALL': 'Learning capacity (L_C_ALL)',
    'Eff_Expl_All': 'Effective exploration ratio (Eff_Expl_All)'
}

# Prepare results container
anova_results = []

# Run factorial ANOVA for each feature
for feature, description in anova_targets.items():
    formula = f"{feature} ~ C(Group) + C(Sex) + C(Year) + C(Group):C(Year) + C(Sex):C(Year)"
    model = ols(formula, data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    anova_table["Feature"] = description
    anova_table["Variable"] = anova_table.index
    anova_results.append(anova_table.reset_index(drop=True))

In [7]:
# Concatenate all results
final_anova_df = pd.concat(anova_results, ignore_index=True)

In [8]:
# Make a copy of the original ANOVA result table
df_anova = final_anova_df.copy()

# Map original variable names to clean labels
label_map = {
    'C(Group)': 'Gr',
    'C(Sex)': 'Sex',
    'C(Year)': 'Year',
    'C(Group):C(Year)': 'Gr/Y',
    'C(Sex):C(Year)': 'Sex/Y'
}

# Filter to include only the relevant ANOVA terms
df_anova = df_anova[df_anova['Variable'].isin(label_map.keys())]

# Create a new column for simplified effect labels
df_anova['Effect'] = df_anova['Variable'].map(label_map)

# Format the F and p-values into a readable string
df_anova['F(p)'] = df_anova.apply(
    lambda row: f"{row['F']:.2f} (p < {row['PR(>F)']:.4f})", axis=1
)

# Pivot the table so each effect is a column and each feature is a row
table_formatted = df_anova.pivot(index='Feature', columns='Effect', values='F(p)')

# Reorder columns to match desired output
ordered_cols = ['Gr', 'Sex', 'Year', 'Gr/Y', 'Sex/Y']
table_formatted = table_formatted.reindex(columns=ordered_cols)

In [9]:
display(table_formatted)

Effect,Gr,Sex,Year,Gr/Y,Sex/Y
Feature,,,,,
Effective exploration ratio (Eff_Expl_All),102.58 (p < 0.0000),3.69 (p < 0.0547),19.32 (p < 0.0000),12.56 (p < 0.0000),3.00 (p < 0.0063)
Exploration (Expl_TOT_Nr),293.82 (p < 0.0000),4.12 (p < 0.0425),41.56 (p < 0.0000),21.92 (p < 0.0000),3.52 (p < 0.0018)
Exploration frequency (Expl_E_I_BEF_Nr),67.93 (p < 0.0000),25.22 (p < 0.0000),34.10 (p < 0.0000),12.19 (p < 0.0000),2.76 (p < 0.0112)
Learning capacity (L_C_ALL),261.45 (p < 0.0000),2.80 (p < 0.0943),28.40 (p < 0.0000),11.74 (p < 0.0000),2.06 (p < 0.0545)
Locomotion (Loco_TOT),147.16 (p < 0.0000),94.49 (p < 0.0000),14.02 (p < 0.0000),12.63 (p < 0.0000),1.70 (p < 0.1163)
Locomotion frequency (Loco_BEF_Calc),37.25 (p < 0.0000),44.34 (p < 0.0000),22.36 (p < 0.0000),2.96 (p < 0.0069),5.35 (p < 0.0000)


In [10]:
# Map feature names to domains based on the given example
domain_map = {
                'Locomotion (Loco_TOT)': 'Activity domain',
                'Locomotion frequency (Loco_BEF_Calc)': 'Activity domain',
                'Exploration (Expl_TOT_Nr)': 'Activity domain',
                'Exploration frequency (Expl_E_I_BEF_Nr)': 'Activity domain',
                'Learning capacity (L_C_ALL)': 'Acquisition domain',
                'Effective exploration ratio (Eff_Expl_All)': 'Error domain'
            }

# Copy the formatted table and add a new column for the domains
formatted_with_domain = table_formatted.copy()
formatted_with_domain['Domain'] = formatted_with_domain.index.map(domain_map)

# Reorder the columns so that "Domain" appears first
cols = ['Domain'] + [col for col in formatted_with_domain.columns if col != 'Domain']
formatted_with_domain = formatted_with_domain[cols]

# Convert the index into a column
formatted_with_domain = formatted_with_domain.reset_index().rename(columns={"index": "Feature"})

# Sort by domain and feature name
formatted_with_domain = formatted_with_domain.sort_values(by=["Domain", "Feature"])

In [11]:
display(formatted_with_domain)

Effect,Feature,Domain,Gr,Sex,Year,Gr/Y,Sex/Y
3,Learning capacity (L_C_ALL),Acquisition domain,261.45 (p < 0.0000),2.80 (p < 0.0943),28.40 (p < 0.0000),11.74 (p < 0.0000),2.06 (p < 0.0545)
1,Exploration (Expl_TOT_Nr),Activity domain,293.82 (p < 0.0000),4.12 (p < 0.0425),41.56 (p < 0.0000),21.92 (p < 0.0000),3.52 (p < 0.0018)
2,Exploration frequency (Expl_E_I_BEF_Nr),Activity domain,67.93 (p < 0.0000),25.22 (p < 0.0000),34.10 (p < 0.0000),12.19 (p < 0.0000),2.76 (p < 0.0112)
4,Locomotion (Loco_TOT),Activity domain,147.16 (p < 0.0000),94.49 (p < 0.0000),14.02 (p < 0.0000),12.63 (p < 0.0000),1.70 (p < 0.1163)
5,Locomotion frequency (Loco_BEF_Calc),Activity domain,37.25 (p < 0.0000),44.34 (p < 0.0000),22.36 (p < 0.0000),2.96 (p < 0.0069),5.35 (p < 0.0000)
0,Effective exploration ratio (Eff_Expl_All),Error domain,102.58 (p < 0.0000),3.69 (p < 0.0547),19.32 (p < 0.0000),12.56 (p < 0.0000),3.00 (p < 0.0063)
